# RFE on raw data

In [5]:
import pandas as pd

## Load Data Set

In [6]:
# Load the data xlsx file as a dataframe
df = pd.read_csv("c:\\Users\\kiera\\OneDrive\\Documents\\GitHub\\dsif-git-main-project\\elvtr_main_project\\data\\raw\\Loan_status_2007-2020Q3\\Loan_status_2007-2020Q3-100k-Full-Data.csv")

# Clean headers in the existing DataFrame 'df'
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# Display cleaned headers
print("Cleaned headers:", df.columns.tolist())

df.shape

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\kiera\\OneDrive\\Documents\\GitHub\\dsif-git-main-project\\elvtr_main_project\\data\\raw\\Loan_status_2007-2020Q3\\Loan_status_2007-2020Q3-100k-Full-Data.csv'

In [ ]:
# Load the data dictionary CSV file as a dataframe
df_data_dict = pd.read_excel("c:\\Users\\kiera\\OneDrive\\Documents\\GitHub\\dsif-git-main-project\\elvtr_main_project\\data\\raw\\Loan_status_2007-2020Q3\\LCDataDictionary.xlsx")

# Clean headers in the existing DataFrame 'df'
df_data_dict.columns = df_data_dict.columns.str.strip().str.lower().str.replace(" ", "_")

# Remove trailing whitespaces in all string columns of df_data_dict
df_data_dict = df_data_dict.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Display cleaned headers
print("Cleaned headers:", df_data_dict.columns.tolist())

df_data_dict.shape

Cleaned headers: ['loanstatnew', 'description']


(153, 2)

In [ ]:
# Essential libraries for data manipulation, statistics, and visualization
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import normaltest, shapiro, anderson, kstest, skew

# Visualization libraries
import matplotlib.pyplot as plt  # For standard plotting
import seaborn as sns  # For static plots with themes
import plotly.express as px  # For interactive plots
import missingno as msno  # For missing data visualization

# Machine Learning libraries
from sklearn.preprocessing import StandardScaler, LabelEncoder, label_binarize
from sklearn.model_selection import train_test_split, cross_val_score  # Data splitting and cross-validation
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import (
    RandomForestClassifier, RandomForestRegressor,
    GradientBoostingClassifier, GradientBoostingRegressor
)
from sklearn.svm import SVC, SVR  # Support Vector Machines for classification and regression
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB  # Naive Bayes Classifier
from sklearn.cluster import KMeans  # K-Means clustering
from sklearn.decomposition import PCA  # Dimensionality reduction
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.feature_selection import RFE  # Recursive Feature Elimination

# Additional machine learning models
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Evaluation metrics
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, roc_curve, precision_recall_curve, average_precision_score
)

# Utility libraries
from tqdm import tqdm
import joblib

# Pandas display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Plot settings for consistent figure size (A4 landscape top half)
FIG_WIDTH = 11.69  # Width
FIG_HEIGHT = 4.14  # Height

# Set the theme for Seaborn plots
sns.set_theme(style='whitegrid')

c:\Users\kiera\AppData\Local\Programs\Python\Python312\Lib\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
def check_infinity(df):
    infinite_list = df.isin([-np.inf, np.inf]).sum()

    if infinite_list.sum() == 0:
        print("No column has infinite values")
    else:
        print("Columns with infinite values:")
        print(infinite_list[infinite_list>0]).sort_values(ascending=False)

check_infinity(df_transformed)

NameError: name 'df_transformed' is not defined

In [ ]:
nan_list = df_transformed.isna().sum()

if nan_list.sum() == 0:
    print("No column has NaN values")
else:
    print("Columns with NaN values (sorted high to low):")
    print(nan_list[nan_list > 0].sort_values(ascending=False))

No column has NaN values


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# 'loan_default' is our target column
X = df
y = df['loan_status_grouped2_kn']
X_columns = X.columns

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Encode the target variable for both training and test sets
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [ ]:
rf = RandomForestClassifier(n_estimators=150, random_state=42) # sets the reproducability to 42
rfe = RFE(estimator=rf, n_features_to_select=48, step=18, verbose=3) # here we're selecting 23 feautures and eleminate 3 features with every iteration of the RFE process.

# Fit RFE on training data only to prevent data leakage
rfe.fit(X_train, y_train)
selected_features = X_train.columns[rfe.support_] # here we're creating a new list with the features the RFE process has captured.

Fitting estimator with 103 features.
Fitting estimator with 85 features.
Fitting estimator with 67 features.
Fitting estimator with 49 features.


We've opted for a random forest classifier, but will evalute select kbest (from sklearn.feature_selection import SelectKBest, f_classif) at a later date.

In [ ]:
# Select the top features
#selected_features = X_train.columns[rfe.support_] # I've opted to keep this for memory purpose.
selected_features_names = list(selected_features)

print("Selected Features by RFE:")
print(f"Index: {selected_features}")
print(f"Column names: {selected_features_names}")

Selected Features by RFE:
Index: Index(['annual_inc', 'bc_open_to_buy', 'dti', 'il_util', 'inq_fi',
       'inq_last_12m', 'inq_last_6mths', 'installment', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_last_delinq', 'mths_since_last_major_derog',
       'mths_since_rcnt_il', 'mths_since_recent_bc',
       'mths_since_recent_bc_dlq', 'mths_since_recent_inq',
       'mths_since_recent_revol_delinq', 'num_actv_bc_tl', 'num_il_tl',
       'open_acc_6m', 'open_act_il', 'percent_bc_gt_75', 'revol_bal',
       'tot_coll_amt', 'total_bal_ex_mort', 'total_cu_tl',
       'total_il_high_credit_limit', 'total_rec_int', 'total_rec_late_fee',
       'total_rec_prncp', 'term_kn', 'emp_length_kn', 'int_rate_kn',
       'revol_util_kn', 'issue_d_year_kn', 'issue_d_month_kn',
       'earliest_cr_line_year_kn', 'earliest_cr_line_month_kn', 'sub_grade_1',
       'hardship_status_kn_1', 'hardship_status_kn_2', 'grade_0', 'grad